<a href="https://colab.research.google.com/github/Kyle950204/KOREA_ZOMBIE-FILM-ANALYSIS/blob/main/%EC%98%81%ED%99%94_%EB%B0%98%EB%8F%84_%EB%A6%AC%EB%B7%B0_%ED%98%95%ED%83%9C%EC%86%8C_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 환경 준비

In [1]:
# konlpy 설치하기
!pip install konlpy

     |████████████████████████████████| 19.4MB 27.9MB/s 
     |████████████████████████████████| 92kB 8.6MB/s 
     |████████████████████████████████| 460kB 41.5MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0


In [2]:
# konlpy를 실행하기
import konlpy

In [3]:
## pandas을 실행하기, 단 실행 별명을 pd로
import pandas as pd

# 데이터 불러오기

In [4]:
# 구글 드라이브 연결을 위한 기본 세팅 
!pip install -U -q PyDrive
 
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [5]:
## 문서 ID로 실제 파일 불러오기
# 반도 30124건.csv
## https://drive.google.com/file/d/1xTgIhcbG-eKuJsa849fb1Grf2AkpvhEB/view?usp=sharing


rawdata_downloaded = drive.CreateFile({'id': '1xTgIhcbG-eKuJsa849fb1Grf2AkpvhEB'})
rawdata_downloaded.GetContentFile('반도 30124건.csv')

In [6]:
# "rawdata.txt" 파일의 내용을 "원본데이터" 변수로 불러오기
원본데이터 = pd.read_csv('반도 30124건.csv', sep=",")

In [7]:
# "원본데이터" 변수 내용 확인하기
원본데이터

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,thsz****,부산행 반도 못따라감 그래서 반도임,2,10042,2740,2020-07-18 19:52:00,관람객
1,1,slow life(ssk8****),재미있게 잘 봤습니다. 다만 조금 질질 끄는 게 아쉬웠어요. 전형적인 약간의 억지 ...,6,5059,1296,2020-07-15 12:07:00,관람객
2,2,로야르(vulc****),진짜 재미없음 다른말필요없음,1,4654,2171,2020-07-15 22:44:00,관람객
3,3,윤등(feel****),영상미는 좋은데 스토리가....흥미가 안생겨요배우님들 고생 많았습니다,4,3424,1131,2020-07-15 12:21:00,관람객
4,4,dlad****,스포일러가 포함된 감상평입니다. 감상평 보기,8,2657,493,2020-07-15 22:25:00,관람객
...,...,...,...,...,...,...,...,...
30115,30115,다시마(junh****),응 잔여 좌석 180석이야~응 개노잼이야~ㅋㅋ,1,21,161,2020-07-15 16:15:00,0
30116,30116,파파라치(taeh****),재밌다재밌다 재미갔다,1,10,191,2020-07-20 09:50:00,0
30117,30117,BFDF(kids****),스포일러가 포함된 감상평입니다. 감상평 보기,1,30,347,2020-07-20 09:35:00,관람객
30118,30118,찡호야(yu_l****),우리나라 영화는 갈수록 후퇴를하네 아...진짜 강동원 잘생긴거말고는 진짜 다쓰레기 ...,2,254,803,2020-07-16 13:15:00,관람객


In [8]:
원본데이터NAN삭제 = 원본데이터.dropna(axis=0)
## 바로: NAN값(빈값)을 삭제하기 위한 프로세스

In [9]:
원본데이터NAN삭제

,Unnamed: 0,nickname,text,score,like,dislike,time,watch_movie
0,0,thsz****,부산행 반도 못따라감 그래서 반도임,2,10042,2740,2020-07-18 19:52:00,관람객
1,1,slow life(ssk8****),재미있게 잘 봤습니다. 다만 조금 질질 끄는 게 아쉬웠어요. 전형적인 약간의 억지 ...,6,5059,1296,2020-07-15 12:07:00,관람객
2,2,로야르(vulc****),진짜 재미없음 다른말필요없음,1,4654,2171,2020-07-15 22:44:00,관람객
3,3,윤등(feel****),영상미는 좋은데 스토리가....흥미가 안생겨요배우님들 고생 많았습니다,4,3424,1131,2020-07-15 12:21:00,관람객
4,4,dlad****,스포일러가 포함된 감상평입니다. 감상평 보기,8,2657,493,2020-07-15 22:25:00,관람객
...,...,...,...,...,...,...,...,...
30115,30115,다시마(junh****),응 잔여 좌석 180석이야~응 개노잼이야~ㅋㅋ,1,21,161,2020-07-15 16:15:00,0
30116,30116,파파라치(taeh****),재밌다재밌다 재미갔다,1,10,191,2020-07-20 09:50:00,0
30117,30117,BFDF(kids****),스포일러가 포함된 감상평입니다. 감상평 보기,1,30,347,2020-07-20 09:35:00,관람객
30118,30118,찡호야(yu_l****),우리나라 영화는 갈수록 후퇴를하네 아...진짜 강동원 잘생긴거말고는 진짜 다쓰레기 ...,2,254,803,2020-07-16 13:15:00,관람객


# 형태소 분석

In [10]:
# konlpy(한국어형태소분석기) 중에서 Okt 실행하기
from konlpy.tag import Okt
okt = Okt()

In [11]:
# "원본데이터NAN삭제"를 대상으로 "okt"를 사용해서 형태소 분석하기
저장공간  = []
for index, row in 원본데이터NAN삭제.iterrows(): # 원본데이터NAN삭제의 컬럼(행)을 불러와서 반복할 준비!
    대상텍스트 = row[2] # 분석에서 사용할 텍스트 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 대상 텍스트의 열은 2번입니다. python은 0부터 숫자를 세게 됩니다!!!
    분류 = row[3] # 분석에서 사용할 분류 정보가 있는 열을 지정해준다. 주의! python은 0부터 시작한다!
    ## 바로: 평점 정보는 3번째 열에 있습니다.
    형태소 = list(okt.pos(대상텍스트)) # okt의 pos 기능을 활용해서 대상 텍스트를 형태소 분리한다.
    형태소종합 = pd.DataFrame.from_records(형태소) # 형태소에 저장된 내용을 데이터프레임 형태로 해서 형태소종합으로 저장한다.
    형태소종합['type'] = 분류 # 형태소종합에 분류정보를 추가한다.
    형태소종합['count'] = 1   # 형태소종합에 카운트정보를 추가한다.   
    저장공간.append(형태소종합) # 형태소종합의 내용을 종합하여 저장한다.

In [12]:
# "형태소" 변수의 저장 내용 확인
형태소

[('정말', 'Noun'),
 ('..', 'Punctuation'),
 ('감동', 'Noun'),
 ('도', 'Josa'),
 (',', 'Punctuation'),
 ('재미', 'Noun'),
 ('도', 'Josa'),
 ('없고', 'Adjective'),
 ('..', 'Punctuation'),
 ('어떻게', 'Adjective'),
 ('좀비', 'Noun'),
 ('영화', 'Noun'),
 ('를', 'Josa'),
 ('이렇게', 'Adverb'),
 ('스릴', 'Noun'),
 ('없이', 'Adverb'),
 ('만들수가', 'Verb'),
 ('있죠', 'Adjective'),
 ('??', 'Punctuation'),
 ('부산', 'Noun'),
 ('행', 'Noun'),
 ('의', 'Josa'),
 ('반의', 'Noun'),
 ('반도', 'Noun'),
 ('못', 'VerbPrefix'),
 ('따라', 'Verb'),
 ('오네요', 'Verb'),
 ('..', 'Punctuation'),
 ('ㅠ', 'KoreanParticle'),
 ('실망', 'Noun'),
 ('입니다', 'Adjective')]

In [13]:
# "형태소종합" 변수의 저장 내용 확인
형태소종합

,0,1,type,count
0,정말,Noun,4,1
1,..,Punctuation,4,1
2,감동,Noun,4,1
3,도,Josa,4,1
4,",",Punctuation,4,1
5,재미,Noun,4,1
6,도,Josa,4,1
7,없고,Adjective,4,1
8,..,Punctuation,4,1
9,어떻게,Adjective,4,1


In [14]:
# "저장공간" 변수의 저장 내용 확인
저장공간

[     0           1  type  count
 0   부산        Noun     2      1
 1    행        Noun     2      1
 2   반도        Noun     2      1
 3    못  VerbPrefix     2      1
 4  따라감        Verb     2      1
 5  그래서      Adverb     2      1
 6   반도        Noun     2      1
 7    임        Noun     2      1,         0            1  type  count
 0    재미있게    Adjective     6      1
 1       잘         Verb     6      1
 2    봤습니다         Verb     6      1
 3       .  Punctuation     6      1
 4      다만         Noun     6      1
 5      조금         Noun     6      1
 6      질질         Noun     6      1
 7      끄는         Verb     6      1
 8       게         Noun     6      1
 9   아쉬웠어요    Adjective     6      1
 10      .  Punctuation     6      1
 11     전형         Noun     6      1
 12      적       Suffix     6      1
 13      인         Josa     6      1
 14     약간         Noun     6      1
 15      의         Josa     6      1
 16     억지         Noun     6      1
 17     감동         Noun     6      1


In [15]:
# "저장공간" 변수의 내용을 데이터프레임 형식으로 변환해서 "분석통합" 변수에 저장한다.
분석통합 = pd.concat(저장공간)

In [16]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,0,1,type,count
0,부산,Noun,2,1
1,행,Noun,2,1
2,반도,Noun,2,1
3,못,VerbPrefix,2,1
4,따라감,Verb,2,1
...,...,...,...,...
26,오네요,Verb,4,1
27,..,Punctuation,4,1
28,ㅠ,KoreanParticle,4,1
29,실망,Noun,4,1


In [17]:
# "분석통합" 변수의 컬럼(열) 이름 변경
분석통합.columns = ["형태소", "품사", "분류", "카운트"]

In [18]:
# "분석통합" 변수의 저장 내용 확인
분석통합

,형태소,품사,분류,카운트
0,부산,Noun,2,1
1,행,Noun,2,1
2,반도,Noun,2,1
3,못,VerbPrefix,2,1
4,따라감,Verb,2,1
...,...,...,...,...
26,오네요,Verb,4,1
27,..,Punctuation,4,1
28,ㅠ,KoreanParticle,4,1
29,실망,Noun,4,1


In [19]:
# "분석통합" 변수의 내용을 형태소, 품사, 분류가 같은 것을 합치고, 카운트의 총합을 구해서 "그룹통합" 변수에 저장한다.
그룹통합 = 분석통합.groupby(['형태소', '품사', "분류"])['카운트'].sum()

In [20]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

형태소  품사           분류
!    Punctuation  1     141
                  2     115
                  3      12
                  4      52
                  5      25
                       ... 
힝    Noun         10      1
힝구   Noun         10      1
힝힝   Noun         10      1
８    Foreign      6       1
～    Foreign      2       1
Name: 카운트, Length: 78461, dtype: int64

In [21]:
# "그룹통합" 변수의 내용을 데이터프레임 형식으로 변환해서 "그룹통합" 변수에 저장한다.
그룹통합 = pd.DataFrame(그룹통합)

In [22]:
# "그룹통합" 변수의 저장 내용 확인
그룹통합

카운트
형태소 품사          분류     
!   Punctuation 1   141
                2   115
                3    12
                4    52
                5    25
...                 ...
힝   Noun        10    1
힝구  Noun        10    1
힝힝  Noun        10    1
８   Foreign     6     1
～   Foreign     2     1

[78461 rows x 1 columns]

In [23]:
# "그룹통합" 변수의 내용을 "형태소분석결과.csv" 파일로 저장한다.
# header는 컬럼(열) 정보의 포함 여부이다.
# encoding은 문자코드를 선택하는 것이다. python에서는 기본적으로 utf-8(유니코드)를 사용한다.

그룹통합.to_csv('형태소분석결과.csv', header='true', encoding='utf-8')